# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('/datasets/users_behavior.csv')
print(data.head())
print()
print(data.info())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


Файл открывается. В данных уверены: "*Предобработка данных не понадобится — вы её уже сделали*". Идем дальше.

## 2. Разбейте данные на выборки

Разделим исходные данные на обучающую, валидационную и тестовую выборки.

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
# Данные делим на 3 группы в пропорции 3:1:1
# для разделения данных на 3 группы дважды применим train_test_split 
data_train, data_valid1 = train_test_split(data, test_size=0.4, random_state=12345)
data_valid, data_test = train_test_split(data_valid1, test_size=0.5, random_state=12345)
print('data_train', round(100 * data_train.shape[0] / data.shape[0]), '%')
print('data_valid', round(100 * data_valid.shape[0] / data.shape[0]), '%')
print('data_test', round(100 * data_test.shape[0] / data.shape[0]), '%')

data_train 60 %
data_valid 20 %
data_test 20 %


## 3. Исследуйте модели

Далее (до тестрования) работаем только с выборками data_train и data_valid

In [6]:
train_features = data_train.drop('is_ultra', axis=1)
train_target = data_train['is_ultra']
valid_features = data_valid.drop('is_ultra', axis=1)
valid_target = data_valid['is_ultra']

Для задачи классификации мы выучили 3 модели:
* дерево решений
* случайный лес
* логистическая регрессия

#### Дерево решений

In [7]:
from sklearn.tree import DecisionTreeClassifier

# В таблице dtc_accuracy для каждого гиперпараметра max_depth будем хранить полученное значение accuracy
dtc_accuracy = pd.DataFrame(columns=['max_depth', 'accuracy'])

# Проверим качество модели для разных значений показатели max_depth (i)
for i in range(1, 42, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model.fit(train_features, train_target)
    predictions = model.predict(valid_features)
    accuracy = model.score(valid_features, valid_target)
    dtc_accuracy.loc[dtc_accuracy.shape[0]] = [i, accuracy]

dtc_accuracy.sort_values(by='accuracy', ascending=False)

,max_depth,accuracy
1,11.0,0.762053
0,1.0,0.754277
2,21.0,0.727838
3,31.0,0.713841
4,41.0,0.713841


Лучшее значение точности получено для гиперпараметра max_depth=11

#### Случайный лес

Для модели "случайный лес" у нас есть дополнительный параметр n_estimators. Создадим отдельную таблицу точности, в которой max_depth также будет меняться от 1 до 41 с шагом 10, и для каждого из значений max_depth гиперпараметр n_estimators также будет проходить от 1 до 41 с шагом 10.

In [8]:
from sklearn.ensemble import RandomForestClassifier

rfc_accuracy = pd.DataFrame(columns=['max_depth', 'n_estimators', 'accuracy'])

# Проверим качество модели для разных значений показатели max_depth (i) и n_estimators (j)
for i in range(1, 42, 10):
    for j in range(1, 42, 10):
        model = RandomForestClassifier(random_state=12345, max_depth=i, n_estimators=j)
        model.fit(train_features, train_target)
        predictions = model.predict(valid_features)
        accuracy = model.score(valid_features, valid_target)
        rfc_accuracy.loc[rfc_accuracy.shape[0]] = [i, j, accuracy]

rfc_accuracy.sort_values(by='accuracy', ascending=False).head(5)

,max_depth,n_estimators,accuracy
6,11.0,11.0,0.794712
14,21.0,41.0,0.794712
22,41.0,21.0,0.793157
17,31.0,21.0,0.793157
13,21.0,31.0,0.791602


Точность модели "случайное дерево" лучше, чем у единичного дерева. Максимальное значение точности предстаказаний было достигнуто для сочетания гиперпараметров max_depth=11, n_estimators=11.

#### Логистическая регрессия

Для логистической регрессии мы не будем подбирать какие-либо параметры. Просто получим accuracy и проверим, не произойдет ли какой-либо сенсации (например, логистическая регрессии по точности опередит случайный лес).

In [9]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=12345)
model.fit(train_features, train_target)
predictions = model.predict(valid_features)
accuracy = model.score(valid_features, valid_target)
print(accuracy)

0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Вывод

Ожидаемо места по точности предсказаний распределились следующим образом:
1. случайный лес
2. решающее дерево
3. логистическая регрессия

**В итоге для проверки на тестовой выборке выбираем модель "случайный лес" с гиперпараметрами max_depth=11, n_estimators=11**

## 4. Проверьте модель на тестовой выборке

In [10]:
test_features = data_test.drop('is_ultra', axis=1)
test_target = data_test['is_ultra']

model = RandomForestClassifier(random_state=12345, max_depth=11, n_estimators=11)
model.fit(train_features, train_target)
predictions = model.predict(test_features)
accuracy = model.score(test_features, test_target)
print(accuracy)

0.7931570762052877


**Требуемая точность (0,75) достигнута!**

## 5. (бонус) Проверьте модели на адекватность

Под адекватностью мы понимаем то, что наша модель лучше случайности. Используем DummyClassifier.

In [11]:
from sklearn.dummy import DummyClassifier

# Проверим точность как если бы каждый раз мы выбирали самое популярное значений (в нашем случае это is_ultra=0)
# Для этого установим гиперпараметр strategy='most_frequent'
dummy_model = DummyClassifier(random_state=12345, strategy='most_frequent')
dummy_model.fit(train_features, train_target)
predictions = dummy_model.predict(test_features)
accuracy = dummy_model.score(test_features, test_target)
print(accuracy)

0.6842923794712286


Также были опробованы другие стратегии для модели DummyClassifier, но получить точность более чем 0,68 не получилось. 

**Таким образом, мы делаем вывод, что наша выбранная модель "случайный лес" адекватно выбирает подходящий для клиента тариф.** 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
